# Creating a data driven network

This example shows how we create and add annotations to a data driven network.

In [1]:
import sys
import os
import networkx as nx
from IPython.display import display
%matplotlib inline

In [2]:
from exp_data import exp_data
from magine.networks.utils import add_attribute_to_network
from magine.networks.network_generator import build_network
from magine.networks.utils import add_data_to_graph

Creating list of seed species and background species for network

In [3]:
measured = exp_data.species.id_list
sig_measured = exp_data.species.sig.id_list
print(len(measured))
print(len(sig_measured))

54818
14426


Now we will create the network. We pass the seed and background list to the network as well as flags turning on all of the network databases. We also trim source/sink nodes (optional). This basically cleans up dangling nodes that are not in our seed or background lists.

In [4]:
network = build_network(
    seed_species=sig_measured,  # seed species
    all_measured_list=measured,  # all data measured
    use_biogrid=True,  # expand with biogrid
    use_hmdb=True,  # expand with hmdb
    use_reactome=True,  # expand with reactome
    use_signor=True,  # expand with signor
    trim_source_sink=True,  # remove all source and sink nodes not measured
    save_name='Networks/bendamustine_network'
)

Trimming network
Network has 21130 nodes and 515061 edges
Found 5475 of 14426 seed species in network
Found 17651 of 54818 background species in network


Now that we have a network generated, we will add attributes to the network. This way we can visualize these attributes.

In [5]:
network = add_data_to_graph(network, exp_data)

save_name = 'bendamustine_network_w_attributes'

# write to GML for cytoscape or other program
nx.write_gml(network, os.path.join('Networks', save_name+'.gml'))

# write to gpickle for fast loading in python
nx.write_gpickle(network, os.path.join('Networks', save_name+'.p'))